# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a Data Frame that loads the csv file exported in Part One
my_cities_df = pd.read_csv("Output/City_Data_Farenheit.csv", encoding="utf-8")

# Print the number of rows in the DataFrame (just to check that it matches the rows exported to the csv file in Part One)
print(len(my_cities_df))

# Display the first 5 rows of the DataFrame
my_cities_df.head()


579


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Kupang,20,ID,1584113216,100,-10.17,123.58,75.20,1.12
1,São Filipe,0,CV,1584112915,54,14.90,-24.50,77.63,15.23
2,Hilo,1,US,1584113198,82,19.73,-155.09,68.00,3.36
3,Letka,97,RU,1584113385,84,59.60,49.42,34.43,9.91
4,Aranda de Duero,84,ES,1584113387,57,41.67,-3.69,61.25,7.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a DataFrame called locations
locations = my_cities_df[["Lat", "Lng"]]

# Create a DataFrame called humidity
humidity = my_cities_df["Humidity"].astype(float)

# Create a variable called for the max humidity
max_humidity = humidity.max()

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create a Data Frame for cities with good weather based on our criteria
my_good_weather_cities_df = my_cities_df.loc[(my_cities_df["Max Temp"] > 70) & (my_cities_df["Max Temp"] < 80) & (my_cities_df["Cloudiness"] == 0) & (my_cities_df["Wind Speed"] < 10), :]

# Drop any rows that do not meet all of our criteria for good weather
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Reset the index for the Data Frame
my_good_weather_cities_df.reset_index(inplace=True)

# Delete the old Index
del my_good_weather_cities_df['index']

# Check the number of rows in the DataFrame
print(len(my_good_weather_cities_df))

# Dispaly the top 5 rows of the DataFrame
my_good_weather_cities_df.head()


17


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hermanus,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
1,Dingle,0,PH,1584113190,76,11.00,122.67,74.39,8.21
2,Mount Isa,0,AU,1584113650,34,-20.73,139.50,78.80,6.93
3,Parola,0,IN,1584113607,16,20.88,75.12,75.29,7.72
4,Pathein,0,MM,1584113873,88,16.78,94.73,75.65,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Iterate through the DataFrame and get the hotel name within the specified parameters for the cities with good weather

for index, row in my_good_weather_cities_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        # "type": "hotel",
        "keyword": "hotel",
        "key": g_key
    }
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json = response.json()
  
    try:
        my_good_weather_cities_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [12]:
# Drop the rows which do not match all the criteria
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Dispaly the top 20 rows of the DataFrame
my_good_weather_cities_df.head(20)




,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Hermanus,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01,The Marine Hermanus
1,Mount Isa,0,AU,1584113650,34,-20.73,139.50,78.80,6.93,Abacus Motel
2,Parola,0,IN,1584113607,16,20.88,75.12,75.29,7.72,Hotel Green Park
3,Pathein,0,MM,1584113873,88,16.78,94.73,75.65,7.31,Pathein Hotel
4,Vangaindrano,0,MG,1584114023,92,-23.35,47.60,74.57,4.61,Hotel Fanilo
5,Alice Springs,0,AU,1584114034,41,-23.70,133.88,78.80,5.82,DoubleTree by Hilton Hotel Alice Springs
6,Luang Prabang,0,LA,1584114148,29,19.89,102.13,76.01,1.50,Avani+ Luang Prabang Hotel
7,Hirara,0,JP,1584114388,84,24.80,125.28,70.61,7.83,HOTEL LOCUS
8,Ormara,0,PK,1584114392,57,25.21,64.64,73.49,5.32,Zero Point Ormara
9,Contai,0,IN,1584114427,61,21.78,87.75,79.97,6.44,Ruposhi Bangla Hotel & Restaurant


In [13]:
my_good_weather_cities_df = my_good_weather_cities_df.reset_index()


In [14]:
my_good_weather_cities_df.head(20)

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,Hermanus,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01,The Marine Hermanus
1,1,Mount Isa,0,AU,1584113650,34,-20.73,139.50,78.80,6.93,Abacus Motel
2,2,Parola,0,IN,1584113607,16,20.88,75.12,75.29,7.72,Hotel Green Park
3,3,Pathein,0,MM,1584113873,88,16.78,94.73,75.65,7.31,Pathein Hotel
4,4,Vangaindrano,0,MG,1584114023,92,-23.35,47.60,74.57,4.61,Hotel Fanilo
5,5,Alice Springs,0,AU,1584114034,41,-23.70,133.88,78.80,5.82,DoubleTree by Hilton Hotel Alice Springs
6,6,Luang Prabang,0,LA,1584114148,29,19.89,102.13,76.01,1.50,Avani+ Luang Prabang Hotel
7,7,Hirara,0,JP,1584114388,84,24.80,125.28,70.61,7.83,HOTEL LOCUS
8,8,Ormara,0,PK,1584114392,57,25.21,64.64,73.49,5.32,Zero Point Ormara
9,9,Contai,0,IN,1584114427,61,21.78,87.75,79.97,6.44,Ruposhi Bangla Hotel & Restaurant


In [16]:
del my_good_weather_cities_df['index']


In [17]:
my_good_weather_cities_df.head(20)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Hermanus,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01,The Marine Hermanus
1,Mount Isa,0,AU,1584113650,34,-20.73,139.50,78.80,6.93,Abacus Motel
2,Parola,0,IN,1584113607,16,20.88,75.12,75.29,7.72,Hotel Green Park
3,Pathein,0,MM,1584113873,88,16.78,94.73,75.65,7.31,Pathein Hotel
4,Vangaindrano,0,MG,1584114023,92,-23.35,47.60,74.57,4.61,Hotel Fanilo
5,Alice Springs,0,AU,1584114034,41,-23.70,133.88,78.80,5.82,DoubleTree by Hilton Hotel Alice Springs
6,Luang Prabang,0,LA,1584114148,29,19.89,102.13,76.01,1.50,Avani+ Luang Prabang Hotel
7,Hirara,0,JP,1584114388,84,24.80,125.28,70.61,7.83,HOTEL LOCUS
8,Ormara,0,PK,1584114392,57,25.21,64.64,73.49,5.32,Zero Point Ormara
9,Contai,0,IN,1584114427,61,21.78,87.75,79.97,6.44,Ruposhi Bangla Hotel & Restaurant


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in my_good_weather_cities_df.iterrows()]
locations = my_good_weather_cities_df[["Lat", "Lng"]]


In [19]:
# Add marker layer on top of heat map
hotel_name = my_good_weather_cities_df["Hotel Name"].tolist()
hotel_city = my_good_weather_cities_df["City"].tolist()
hotel_info = [hotel_name, hotel_city]
locations = my_good_weather_cities_df[["Lat", "Lng"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=[f"Name: {name}" for name in hotel_info[0]])

# Add the layer to the map
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))